# Solving Frozen Lake With Q-Learning

In this notebook we'll implement q-learning on a simple discrete environment called Frozen Lake. Here is what the documentation says about Frozen Lake:

> Winter is here. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend.

> The surface is described using a grid like the following:

```
SFFF       (S: starting point, safe)
FHFH       (F: frozen surface, safe)
FFFH       (H: hole, fall to your doom)
HFFG       (G: goal, where the frisbee is located)
```

> The episode ends when you reach the goal or fall in a hole. You receive a reward of 1 if you reach the goal, and zero otherwise.

The documentation on OpenAI is infuriatingly sparse, but the environments and API is incredibly helpful. Get used to digging around in the [source code](https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py) if you want to fully understand the environments. For example, the source indicates which actions are which:

```
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
```

And this is what they mean when they say the "ice is slippery":

```python
# ...
# Code above is nested for loops to address each state/action pair 
if is_slippery:
    for b in [(a-1)%4, a, (a+1)%4]:
        newrow, newcol = inc(row, col, b)
        newstate = to_s(newrow, newcol)
        newletter = desc[newrow, newcol]
        done = bytes(newletter) in b'GH'
        rew = float(newletter == b'G')
        li.append((1.0/3.0, newstate, rew, done))
else:
    newrow, newcol = inc(row, col, a)
    newstate = to_s(newrow, newcol)
    newletter = desc[newrow, newcol]
    done = bytes(newletter) in b'GH'
    rew = float(newletter == b'G')
    li.append((1.0, newstate, rew, done))
```

This line is the most important one: `[(a-1)%4, a, (a+1)%4]`. It indicates that taking an action might result in one of three actions being taken, and what those actions could be. Consider `left=0`: you have a 1 in 3 chance each of actually going left, up, or down. Because:

```
0-1 % 4 = 3 => UP
0+1 % 4 = 1 => DOWN
```

When the ice is slipperty every action might send you in one of the two perpindicular directions. Specifically:

```
LEFT  ->  LEFT, UP, DOWN
RIGHT ->  RIGHT, UP, DOWN
UP    ->  UP, LEFT, RIGHT
DOWN  ->  DOWN, LEFT, RIGHT
```

This is critical knowledge because it dramatically impacts the optimal policy. 

## Q-Learning

Q-learning is an algorithm that iteratively explores an environment in order to experimentally approximate the Bellman equation: 

![](images/q-bellman.png)

With Q-Learning, we have our agent switch between "exploring" the state space and "exploiting" the optimal policy (at least, our current notion of that policy). The below is an example of an "epsilon greedy" Q-learning algorithm, which means some percent of the time our agent picks an action at random, and the rest of the time it picks whatever our current "optimal" action is, defined by the Q-Table. We define an "exploration rate" to control how often the agent explores at random, this exploration rate is often denoted by the symbol epsilon (ε), hence "epsilon greedy".

Every time our agent takes an action, we will update our Q-Table, which contains an entry for every state-action pair that our agent encounters. We initialize this table to be 0 for all state-action pairs. Additionally, Q-Learning introduces a "learning rate" which is similar to the learning rate in Stochastic Gradient Descent. 

In [56]:
# Before we do the harder, stocastic, version of this problem
# Lets appreciate how fast Q-Learning converges on an optimal 
# policy when the action's it takes are always respected.

# Initialize the environment, set is_slippery to false.
environment = gym.make('FrozenLake-v0', is_slippery=False)

# The Q-Table is a map of state-action pairs to the estimated value of 
# that state-action pair. In this game there are 16 states and 4 actions
# per state. This simple 2D array captures all the state-action pairs 
# nicely. And we don't know anything about the values yet so we use 0
# as the initial value for all state-action pairs. 
q_table = np.zeros([environment.observation_space.n, environment.action_space.n])

# Some global parameters for Q-Learning.
# These are like network hyperparameters, they might change per task
# and we might even change them during the training episodes.
learning_rate = 0.01 
discount_factor = 0.95
exploration_rate = 0.3

# Values to constrain training, sometimes we might train until we 
# detect policy convergance, sometimes we'll train a fixed number
# of episodes. Right now we're doing the latter. 
training_episodes = 10000

# To help us evaluate the performance
episodes_per_evaluation = 500

for current_episode_num in range(training_episodes):
    state = environment.reset()
    done = False
    
    while not done:
        # First decide if we're taking a random action or the current
        # most valuable action.
        explore = np.random.random() < exploration_rate

        if explore:
            action = environment.action_space.sample()
        else:
            # If we're not exploring randomly, we need to examine the Q-table 
            # to determine the best possible action given the current state
            action = np.argmax(q_table[state])

        # Regardless of how we got the action, take it!
        next_state, reward, done, _ = environment.step(action)

        # Based on the reward we got for taking that action
        # update the Q-table entry for the pair (state, action)
        # NOTE: we are NOT updating the value for next_state
        # Further note, I've split what is often represented in one 
        # formula across a couple lines for readability
        prev_q_value = q_table[state, action]
        discounted_future_reward = discount_factor * np.max(q_table[next_state])

        q_table[state, action] = (
            prev_q_value + (learning_rate * (reward + discounted_future_reward - prev_q_value))

        )
        
        # Update the state for the next round.
        # CRITICAL, don't forget this!
        state = next_state
    
    # Every so often try 100 games and see the average reward. 
    if current_episode_num % episodes_per_evaluation == 0:
        rew_average = 0.
        for i in range(100):
            obs = environment.reset()
            done = False
            while done != True: 
                action = np.argmax(q_table[obs])
                obs, rew, done, info = environment.step(action) #take step using selected action
                rew_average += rew
        rew_average=rew_average/100
        print(f'Episode {current_episode_num} avarage reward: {rew_average}')
        
        # 0.8 is considered solved since 20% of the time the agents
        # move is ignored which can cause us to fall into a lake
        # So we'll break here with an "optimal" policy.
        if rew_average > 0.8:
            break



Episode 0 avarage reward: 0.0
Episode 500 avarage reward: 0.0
Episode 1000 avarage reward: 0.0
Episode 1500 avarage reward: 1.0


In [58]:
# We quickly learn to play the game perfectly.
# Which is not surprising, we could have just
# used graph search to be honest. 

# Still, it's informative to look at the Q-table and visualize the policy
print(q_table)

# This is the board:

# SFFF       (S: starting point, safe)
# FHFH       (F: frozen surface, safe)
# FFFH       (H: hole, fall to your doom)
# HFFG       (G: goal, where the frisbee is located)

actions=['<', '∨', '>', '∧']
policy_string = ''

for location_id, q_values in enumerate(q_table):
    if location_id % 4 == 0: 
        policy_string += '\n'
    idx = np.argmax(q_values)
    policy_action = actions[idx]
    policy_string += policy_action

print(policy_string)
print('''
SFFF 
FHFH 
FFFH 
HFFG 
''')

[[7.36742066e-03 8.58158699e-02 7.90981508e-04 8.80274393e-03]
 [8.34693625e-03 0.00000000e+00 6.04772155e-07 8.79286961e-05]
 [5.38116595e-05 1.07573947e-03 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.01454898e-02 1.60588692e-01 0.00000000e+00 8.61722404e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 6.25651648e-02 0.00000000e+00 5.13543969e-06]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.40081506e-02 0.00000000e+00 2.81357613e-01 1.24308323e-02]
 [1.69223492e-02 1.73690977e-02 4.64264220e-01 0.00000000e+00]
 [4.27959333e-02 7.00260734e-01 0.00000000e+00 3.73628074e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.71102569e-03 2.05746983e-01 3.43170713e-03]
 [1.28387943e-02 1.70366701e-01 9.25951647e-01 7.99297740e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.000000

In [59]:
# Initialize the environment, note that we are not using the monitor here
# because we can train much faster if we don't render anything. 
environment = gym.make('FrozenLake-v0', is_slippery=True)

# The Q-Table is a map of state-action pairs to the estimated value of 
# that state-action pair. Being in a state very close to the winning state
# and taking an action that leads to victory should be highly valued
# whereas being in a state where you're about to lose and taking an 
# action that leads to a loss should have a low value. But before 
# we know anything, we just set all the values to 0. 
q_table = np.zeros([environment.observation_space.n, environment.action_space.n])

# Some global parameters for Q-Learning
learning_rate = 0.01 
discount_factor = 0.95
exploration_rate = 0.3

# Values to constrain training
# Note the very large number...
training_episodes = 200000

# To help us evaluate the performance
episodes_per_evaluation = 5000

for current_episode_num in range(training_episodes):
    state = environment.reset()
    done = False
    
    while not done:
        # First decide if we're taking a random action or the current
        # most valuable action.
        explore = np.random.random() < exploration_rate

        if explore:
            action = environment.action_space.sample()
        else:
            # If we're not exploring randomly, we need to examine the Q-table 
            # to determine the best possible action given the current state
            action = np.argmax(q_table[state])

        # Regardless of how we got the action, take it!
        next_state, reward, done, _ = environment.step(action)

        # Based on the reward we got for taking that action
        # update the Q-table entry for the pair (state, action)
        # NOTE: we are NOT updating the value for next_state
        # Further note, I've split what is often represented in one 
        # formula across a couple lines for readability
        prev_q_value = q_table[state, action]
        discounted_future_reward = discount_factor * np.max(q_table[next_state])

        q_table[state, action] = (
            prev_q_value + (learning_rate * (reward + discounted_future_reward - prev_q_value))

        )
        
        # Update the state for the next round.
        # CRITICAL, don't forget this!
        state = next_state
    
    # Every so often try 100 games and see the average reward. 
    if current_episode_num % episodes_per_evaluation == 0:
        #report every 5000 steps, test 100 games to get avarage point score for statistics and verify if it is solved
        rew_average = 0.
        for i in range(100):
            obs = environment.reset()
            done = False
            while done != True: 
                action = np.argmax(q_table[obs])
                obs, rew, done, info = environment.step(action) #take step using selected action
                rew_average += rew
        rew_average=rew_average/100
        print(f'Episode {current_episode_num} avarage reward: {rew_average}')
        
        # 0.8 is considered solved since somtimes the agents
        # move is ignored which can cause us to fall into a lake
        # So we'll break here with an "optimal" policy.
        if rew_average > 0.8:
            break



Episode 0 avarage reward: 0.0
Episode 5000 avarage reward: 0.07
Episode 10000 avarage reward: 0.39
Episode 15000 avarage reward: 0.76
Episode 20000 avarage reward: 0.78
Episode 25000 avarage reward: 0.72
Episode 30000 avarage reward: 0.77
Episode 35000 avarage reward: 0.76
Episode 40000 avarage reward: 0.77
Episode 45000 avarage reward: 0.75
Episode 50000 avarage reward: 0.72
Episode 55000 avarage reward: 0.74
Episode 60000 avarage reward: 0.74
Episode 65000 avarage reward: 0.74
Episode 70000 avarage reward: 0.78
Episode 75000 avarage reward: 0.65
Episode 80000 avarage reward: 0.73
Episode 85000 avarage reward: 0.72
Episode 90000 avarage reward: 0.7
Episode 95000 avarage reward: 0.86


In [60]:
# Hmmmm now we learn quite slowly, and never get a
# perfect score.
print(q_table)

# This is the board:

# SFFF       (S: starting point, safe)
# FHFH       (F: frozen surface, safe)
# FFFH       (H: hole, fall to your doom)
# HFFG       (G: goal, where the frisbee is located)

actions=['<', '∨', '>', '∧']
policy_string = ''

for location_id, q_values in enumerate(q_table):
    if location_id % 4 == 0: 
        policy_string += '\n'
    idx = np.argmax(q_values)
    policy_action = actions[idx]
    policy_string += policy_action

print(policy_string)
print('''
SFFF 
FHFH 
FFFH 
HFFG 
''')

[[0.17863465 0.17529071 0.17665382 0.16806531]
 [0.11126537 0.10709323 0.10223171 0.16080495]
 [0.15573996 0.14947138 0.14955051 0.14325015]
 [0.09007912 0.09374907 0.08560954 0.13760651]
 [0.20404427 0.16032428 0.15207944 0.11783304]
 [0.         0.         0.         0.        ]
 [0.17366802 0.1141102  0.16626483 0.0509508 ]
 [0.         0.         0.         0.        ]
 [0.14665247 0.20041093 0.16856191 0.26760817]
 [0.26725309 0.36542711 0.30874359 0.20794106]
 [0.3914147  0.33670748 0.28983959 0.18708371]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.2908254  0.41138156 0.51232013 0.36006388]
 [0.5265021  0.7570494  0.68815999 0.63438137]
 [0.         0.         0.         0.        ]]

<∧<∧
<<<<
∧∨<<
<>∨<

SFFF 
FHFH 
FFFH 
HFFG 



In [62]:
# Does that policy table make sense to you?
# Why doesn't this match the policy we saw before?

# I would agrue the AI is actually behaving very rationally here...

# With neural networks, after training we had a trained model
# In Q-Learning after training we have a policy that can be followed
# by an agent. We can use the agent by removing the exploration factor
# and not updating the values in the q-table.

# Lets visualize a single playthrough.
state = environment.reset()
for _ in range(max_steps_per_episode):

    action = np.argmax(q_table[state, :])
    state, reward, done, _ = environment.step(action)
    environment.render()
    
    # If the game finished before our max number of rounds, break out
    if done: break

  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH

In [ ]:
# The AI always chooses an action that ensures it won't fall in a hole!!